# Домашнее задание № 10. Генерация текста/

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 46.9 MB/s 
     |████████████████████████████████| 6.6 MB 17.3 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [ ]:
# возьмем модель поменьше, так как дообучение это обновление весов
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
texts = []

with open("anek.txt", "r") as f:
  for text in f.readlines():
    # delete <|startoftext|> metainfo
    text = text.replace("<|startoftext|>", "")
    if len(text) > 3:
       texts.append(text)


In [ ]:
len(texts)

124166

In [ ]:
# Get only 5000 examples for reduce train time
texts = texts[:5000]

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write("\n".join(texts))

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 2141
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 400


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=400, training_loss=3.2197439575195315, metrics={'train_runtime': 6487.7371, 'train_samples_per_second': 33.001, 'train_steps_per_second': 0.062, 'total_flos': 6989790486528000.0, 'train_loss': 3.2197439575195315, 'epoch': 99.96})

In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет  до небес, - говорит, глядя в маленькое зеркальце, мэр города Нотр-Дам.

Учебник русского матрешки:- Итак ребята, на чем вы остановились, когда вам дали двойку?

Ходил тут Трамп по американскому пляжу, пел серенаду перед всеми отдыхающими. Никто из отдыхающих до Путина не долетел. Трамп рассмеялся и пошел своей дорогой.

Жена с дочкой возвращаются со школы. Одна из дочерей рассказывает учительнице:- Мам, а почему я не сняла джинсы, что бы мама не знала? Она бы меня спросила. И куда бы еще одевать ее платьица?

После длительного карантина Трамп пошел на концерт в поддержку Клинтон. Сидят дети у компьютера у главного входа в парк и громко пою. Под конец один мальчик заплакал и спрашивает:- Мамочка, а папе самому пойти с ней поплакать?

Ложь опаснее всего, когда тебе кто-то


Можно сделать вывод, что анекдоты для нейросети сложны.

### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

1) Статьи:
- GPT-1: Radford, A., Narasimhan, K., Salimans, T., & Sutskever, I. (2018). Improving language understanding by generative pre-training.
- GPT-2: Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). Language models are unsupervised multitask learners.
- GPT-3: Brown, T., Mann, B., Ryder, N., Subbiah, M., Kaplan, J. D., Dhariwal, P., ... & Amodei, D. (2020). Language models are few-shot learners.

2) Обучающий корпус состоял из 2 непересекающихся корпусов книг, статей из википедии, датасета WebText и датасета Common Crawl.

Авторы обнаружили, что неотфильтрованные или слабо отфильтрованные версии Common Crawl имеют низкое качество, поэтому они предприняли три шага с целью улучшения качества данных: (1) авторы отфильтровали версию CommonCrawl на основе сходства с другими высококачественными reference корпусами, (2) сделали fuzzy deduplication на уровне документа, внутри и между наборами данных, чтобы предотвратить избыточность и сохранить целостность held-out валидационного набора в качестве достоверной оценки переобучения, и (3) добавили известные высококачественные reference корпуса в обучающую выборку, чтобы увеличить существующий CommonCrawl и повысить его разнообразие.